In [1]:
import geopy.distance
import csv
import math

In [2]:
# File name/path
file = 'gpsData.csv'

In [3]:
# Define constants
R = 6371  # Earth Mean radius in Km

In [4]:
data_len = 5
lat_A = 1
lat_B = 3
lon_A = 2
lon_B = 4

In [9]:
"""First Method"""

# We'll use the Haversine method to calculate the "as-the-crow-flies"
def haversine(phi1, phi2, l1, l2):
    
    d_phi = phi1 - phi2
    d_l = l1 - l2
    a = math.sin(d_phi / 2) * math.sin(d_phi / 2) + math.cos(phi1) * math.cos(phi2) * math.sin(d_l / 2) * math.sin(
        d_l / 2)

    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

with open(file, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if len(row) == data_len:
            row = [(math.pi / 180) * float(x) for x in row]
            print(haversine(row[lat_A], row[lat_B], row[lon_A], row[lon_B]))

1557.8511885864168
1783.1837464043367
737.7727660154981
3749.773736547801
3675.982245031006
967.1798170292865
3274.1417287219074
1316.846303196569
1334.5937358645376
1136.5465725367428
3114.531417861335
787.5725816616357
2643.0580378691548
2464.362753912112
489.86692923676503
2062.449831897548
1128.2596197590133
2073.259555863112
623.6154918784914
2565.229426430388
2985.2313244972984
1214.251052013342
664.4137001222578
1092.7165524462378
842.8853572563844
1037.5512974092615
1188.4372800292472
1074.0735663346675
1673.2795893889559
1047.25217821814
1555.1616547969786
1422.246835217927
443.0514907503302
1000.1429923016462
1822.1040541639297
217.2294576425259
876.9104589502133
3847.2269875248826
1210.6961872938382
1138.0246799805477
1005.2236284656058
2082.3223407333558
1595.508945921367
1257.4567004158644
197.8526985282972
1104.429834660124
1757.9709021093704
2326.319076338545
1011.3946000387128
712.4302559494926
1884.4567643866728
886.4921431947865
904.2090058156622
1336.9954879608038
56

In [7]:
"""Second Method"""

with open(file, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if len(row) == data_len:
            coord_A = (float(row[lat_A]), float(row[lon_A]))
            coord_B = (float(row[lat_B]), float(row[lon_B]))

            print(geopy.distance.distance(coord_A, coord_B).km)

1558.8537301904857
1780.2974858648208
737.8706376262918
3759.1778759602375
3683.1733469266187
969.248759027644
3280.6491174497196
1314.7640234157784
1334.3124824453287
1134.6952321772912
3119.7375161117357
789.8319571733576
2649.9083903207274
2464.918998250112
490.44491408602863
2066.4805831666163
1131.220044417168
2071.8573519871225
624.1310114434868
2571.5716654897046
2993.116075945281
1211.9755243617028
665.5032611864912
1090.8157192144188
844.8910763068379
1036.5309530002442
1189.5123163471376
1074.3893259696001
1671.621381389165
1045.197842112364
1558.3032764652435
1425.7250484261876
443.6871430646663
1002.2066984629232
1822.7903602404067
217.59635334624582
877.5840618351172
3856.114480129037
1213.5750659646924
1136.0665040891408
1007.8955363209233
2087.665263774418
1596.6938253518974
1259.8647859240614
197.85850388372475
1102.6989294507323
1757.4509032725266
2330.86062315961
1012.0993159194335
713.9559301479616
1889.0389072778553
888.4758958950956
905.5455116244043
1339.794760908